In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [2]:
%cd "/content/gdrive/MyDrive/Text Matching/"

/content/gdrive/MyDrive/Text Matching


In [3]:
import jieba
from sklearn.model_selection import StratifiedShuffleSplit
import pandas as pd
import numpy as np
import random

In [4]:
Data = pd.read_csv("clean_data")

In [5]:
def jieba_tokenize(sentence):
    return " ".join([word for word in jieba.cut(sentence)])

Data["语义1"] = Data["语义1"].apply(jieba_tokenize)
Data["语义2"] = Data["语义2"].apply(jieba_tokenize)
Data.head()

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.791 seconds.
Prefix dict has been built successfully.


,搜索ID,语义1,语义2,匹配与否
0,0,事故 住 建厅 通报 福建 一 工地 发生 坍塌 事故 致 死伤,突发 广元 一 隧道 内 发生 三 车 相撞 事故 致 死伤,0
1,1,抢险 牺牲 山东 莒县 村干部 孟凡勇 没 来得及 和 家人 拍 全家福,利奇马 席卷 山东 铝企 抢险 实况,0
2,2,张云雷刚 退赛 欢乐 喜剧 人 就 贴 上 歌手 标签 要 转行 的 节奏,欢乐 喜剧 人 我 是 歌手,1
3,3,这个 台湾省 爱国 教育 基地 被 强拆 前 再 一次 升起 了 五星红旗,台湾 升起 五星红旗,1
4,4,微博能 发张 图片 了 想 晒 库存 仅 对 粉丝 数 大于 万 的 用户 开放,发 一个 库存 喜欢 女儿 笑 你 笑 起来 真 好看,0


In [6]:
Data['语义1'] = Data['语义1'].replace('', np.nan)
Data["语义2"] = Data["语义2"].replace('', np.nan)
Data.dropna(subset=['语义1'],inplace=True)
Data.dropna(subset=["语义2"],inplace=True)

### 将数据集分为训练集，验证集和测试集

In [11]:
data_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_data = None
val_test_data = None
for train_index, val_test_index in data_split.split(Data, Data["匹配与否"]):
    train_data = Data.iloc[train_index,:]
    val_test_data = Data.iloc[val_test_index,:]
print("shape of train_data: ", train_data.shape)
print("shape of val_test_data: ", val_test_data.shape)

shape of train_data:  (538446, 4)
shape of val_test_data:  (134612, 4)


In [13]:
data_split = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
valid_data = None
test_data = None
for valid_index, test_index in data_split.split(val_test_data, val_test_data["匹配与否"]):
    valid_data = Data.iloc[valid_index,:]
    test_data = Data.iloc[test_index,:]
print("shape of valid_data: ", valid_data.shape)
print("shape of test_data: ", test_data.shape)

shape of valid_data:  (67306, 4)
shape of test_data:  (67306, 4)


### 生成MatchNet输入格式数据集

In [15]:
data_list = train_data.values.tolist()
data_list = [[str(data[0]),data[1],data[2],"",str(data[3])] for data in data_list]
data_list[:5]
matchnet_data_list = ['\t'.join(data) for data in data_list]
matchnet_data_list[:5]

['118064\t家长 给 抢报 保洁 岗位 的 孩子 写 了 一封信 校长 看 了 立即 回信 点赞\t给 校长 的 一封信\t\t1',
 '456275\t伊朗 名宿 日本 是 亚洲杯 热门 伊日 或 相遇 决赛\t日本 伊朗 亚洲杯\t\t1',
 '340222\t肿瘤 免疫治疗 备受 关注 肺癌 有望 成为 慢性 疾病\t肺癌 肿瘤 免疫治疗\t\t1',
 '182592\t李大霄 又 双 叒 叕 要 退出 江湖 了 还 说 听 网友 意见 嫩 啥 意见\t李大霄 要 退出 江湖\t\t1',
 '294493\t李雲天 塔罗牌 处女座 年 下半年 星座 运势\t年 处女座 星座 运势\t\t1']

In [16]:
with open("train.txt", "w") as outfile:
  for unit_data in matchnet_data_list:
    outfile.write(unit_data)
    outfile.write('\n') 

In [17]:
data_list = valid_data.values.tolist()
data_list = [[str(data[0]),data[1],data[2],"",str(data[3])] for data in data_list]
data_list[:5]
matchnet_data_list = ['\t'.join(data) for data in data_list]
matchnet_data_list[:5]

['7128\t同 城 姚劲波 乡镇 二手车 交易 容易 打动 下沉 市场 用户\t二手车 交易市场 同 城\t\t1',
 '29162\t福布斯 球队 价值 排行榜 出炉 美国 之队 十二 连庄\t美国 福布斯 排行榜\t\t1',
 '54952\t上海 堡垒 就 侵权 道歉 了 称 是因为 工作 疏忽 未经 授权 就 使用 了 原博主 的 作品 已经 第一 时间 下架 将 主动 赔偿 原博主\t这位 时尚 博主 活力 爆表 了 哪来 的 神仙 激励 法\t\t0',
 '76909\t冯绍峰 和 赵丽颖 在 香港 购物 被 偶遇 这次 终于 是 正面 照 了\t冯绍峰 和 赵丽颖 在 香港\t\t1',
 '110246\t为 学子 祈福 哈尔滨 极乐寺 挂 升学 牌通启\t哈尔滨 哈尔滨 极乐寺\t\t1']

In [18]:
with open("val.txt", "w") as outfile:
  for unit_data in matchnet_data_list:
    outfile.write(unit_data)
    outfile.write('\n') 

In [19]:
data_list = test_data.values.tolist()
data_list = [[str(data[0]),data[1],data[2],"",str(data[3])] for data in data_list]
data_list[:5]
matchnet_data_list = ['\t'.join(data) for data in data_list]
matchnet_data_list[:5]

['132322\t醉驾 上路 被查获 留下 案底 悔恨 不已\t无罪判决 隔夜 醉驾 宣告无罪\t\t0',
 '106397\t印度 富商 要求 植发 根植 了 根后 就 不治 身亡\t富商 要求 植发\t\t1',
 '13785\t中央 广播电视 总台 农业 农村 频道 开播\t中央 电视 农业 频道\t\t1',
 '60186\t她 是 冯巩 女 搭档 被 称为 小品 公主 却 因患 抑郁症 淡出 荧屏\t小品 公主 冯巩 搭档\t\t1',
 '125753\t男子 为 泄愤 楼 扔下 刀具 以 危害 公共安全 罪 获刑 一年\t如果 长江 实业 以 敲诈勒索罪 诬告 员工 会 如何\t\t0']

In [20]:
with open("test.txt", "w") as outfile:
  for unit_data in matchnet_data_list:
    outfile.write(unit_data)
    outfile.write('\n') 